In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from preprocessing import preprocessing
from features_engineering import augment
from ml_methods import *
from proj1_helpers import *
import pickle as pkl

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
DATA_TRAIN_PATH = '../data/train.csv' 
DATA_TEST_PATH = '../data/test.csv'
y, tX_train, ids = load_csv_data(DATA_TRAIN_PATH)
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [3]:
(xs_train, masks_train) = preprocessing(tX_train)
(xs_test, masks_test) = preprocessing(tX_test)

In [4]:
y_submission = np.zeros(tX_test.shape[0])
for x_train, mask_train, x_test, mask_test in zip(xs_train, masks_train, xs_test, masks_test):
    print("#######New subset#######")
    y_correspond = y[mask_train]
    x_train_aug = augment(x_train, 3)
    print("Augmented train")
    del x_train
    w,_ = least_squares(y_correspond, x_train_aug)
    print("Computed weights")
    del x_train_aug
    x_test_aug = augment(x_test, 3)
    print("Augmented test")
    del x_test
    y_submission[mask_test] = predict_labels(w, x_test_aug)
    print("Computed predictions")
    del x_test_aug
print(y_submission)

#######New subset#######
Augmented train
Computed weights
Augmented test
Computed predictions
#######New subset#######
Augmented train
Computed weights
Augmented test
Computed predictions
#######New subset#######
Augmented train
Computed weights
Augmented test
Computed predictions
[-1.  1. -1. ...  1. -1. -1.]


## Generate predictions and save ouput in csv format for submission:

In [5]:
OUTPUT_PATH = '../predictions.csv' # TODO: fill in desired name of output file for submission
# y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_submission, OUTPUT_PATH)